In [1]:
import os
import sys
from tqdm import tqdm
from PIL import Image
import numpy as np

sys.path.extend(['..'])

from utils.config import process_config

import tensorflow as tf
from tensorflow.layers import (conv2d, max_pooling2d, average_pooling2d, batch_normalization, dropout, dense)
from tensorflow.nn import (relu, sigmoid, softmax, )

from sklearn.utils import shuffle

In [2]:
config_tf = tf.ConfigProto(allow_soft_placement=True)
config_tf.gpu_options.allow_growth = True
config_tf.gpu_options.per_process_gpu_memory_fraction = 0.95

In [3]:
DATA = '../data/data_clean/'
CONF = '../configs/roman.json'

In [4]:
config = process_config(CONF)
config

{'exp_name': 'data',
 'seed': 230,
 'image_size': 128,
 'train_percentage': 0.8,
 'summary_dir': '../experiments/data/summary/',
 'checkpoint_dir': '../experiments/data/checkpoint/'}

In [5]:
def normalize(image):
    return (image - image.min()) / (image.max() - image.min())

def shuffle_sim(a, b):
    assert a.shape[0] == a.shape[0], 'Shapes must be equal'
    
    ind = np.arange(a.shape[0])
    np.random.shuffle(ind)
    return a[ind], b[ind]

In [6]:
def read_train_test(path_to_data):
    data = {}
    for dset in ['train', 'test']:
        path_ = os.path.join(path_to_data, dset)
        X, Y = [], []
        classes = [d for d in os.listdir(path_) if os.path.isdir(os.path.join(path_, d))]
        classes.sort()
        
        for cl in classes:
            y = np.zeros((1, 8), dtype=np.int32)
            y[0, int(cl) - 1] = 1
            
            cl_path = os.path.join(path_, cl)
            filenames = [os.path.join(cl_path, pict) for pict in os.listdir(cl_path) if pict.endswith('.jpg')]
            
            for im in filenames:
                image = np.asarray(Image.open(im), dtype=np.float32)
                X.append(normalize(image).reshape((1, image.shape[0], image.shape[1], image.shape[2])))
                Y.append(y)
        
        a, b = shuffle_sim(np.concatenate(X), np.concatenate(Y))
        data[dset] = ([a, b])
    return data

In [7]:
EPOCHS = 100
LR = 5e-2

In [ ]:
class Model():
    
    def __init__(self, config, sess_cf, learning_rate):
        self.lr = learning_rate
        self.sess = tf.Session(config=sess_cf)

        self.x = tf.placeholder(dtype=tf.float32, shape=(None, config.image_size, config.image_size, 3))
        self.y = tf.placeholder(dtype=tf.int32, shape=(None, 8))
        self.training = tf.placeholder(dtype=tf.bool, shape=())

        global_step = tf.Variable(1, name='global_step', trainable=False, dtype=tf.int32)
        self.step = tf.assign(global_step, global_step + 1)
        
        self.model()
        
        self.summ_writer = tf.summary.FileWriter(config.summary_dir, graph=self.sess.graph)
        self.sess.run(tf.global_variables_initializer())
    
    def block(self, inp,
              ch,
              num,
              c_ker=(3, 3),
              c_str=(1, 1),
              act=relu,
              mp_ker=(2, 2),
              mp_str=(2, 2)):
    
        with tf.variable_scope('block_' + str(num)):
            conv = conv2d(inp, ch, c_ker, strides=c_str)
            bn = batch_normalization(conv)
            out = act(bn)
            tf.summary.histogram('conv1', conv)
            print(out.shape)
            
            conv = conv2d(out, ch, c_ker, strides=c_str)
            bn = batch_normalization(conv)
            out = act(bn)
            tf.summary.histogram('conv2', conv)
            print(out.shape)
            
            out = max_pooling2d(out, (4, 4), strides=(2, 2))
            print(out.shape)
        return out
    
    def model(self):
        with tf.name_scope('layers'):
            out = self.block(self.x, 32, 1)
            out = self.block(out, 64, 3, c_ker=(5, 5), c_str=(2, 2))
            
            dim = np.prod(out.shape[1:])
            out = tf.reshape(out, [-1, dim])
            print(out.shape)
            
            dense_l = dense(out, 128, activation=relu)
            tf.summary.histogram('dense', dense_l)
            out = dropout(dense_l, rate=0.6, training=self.training)
            print(out.shape)

            self.predictions = dense(out, 8, activation=softmax)
            tf.summary.histogram('pred', self.predictions)

        with tf.name_scope('metrics'):    
            amax_labels = tf.argmax(self.y, 1)
            amax_pred   = tf.argmax(self.predictions, 1)

            self.loss = tf.losses.softmax_cross_entropy(self.y, self.predictions)        
            self.acc = tf.reduce_mean(tf.cast(tf.equal(amax_labels, amax_pred), dtype=tf.float32))

            self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.loss)

            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.acc)

        self.summary = tf.summary.merge_all()
        
    def train(self, dat, dat_v, epochs):
        for epoch in range(epochs):
            loss, acc, _, summary, step = self.sess.run([
                self.loss, self.acc, self.optimizer, self.summary, self.step
            ],
                                                feed_dict={
                                                    self.x: dat[0],
                                                    self.y: dat[1],
                                                    self.training: True
                                                })

            self.summ_writer.add_summary(summary, step)
            print('EP: {:3d}\tLOSS: {:.10f}\tACC: {:.10f}'.format(
                epoch, loss, acc))

            if epoch % 10 == 0 and epoch != 0:
                self.test(dat_v)
                
    def test(self, dat):
        loss, acc = self.sess.run([self.loss, self.acc],
                                         feed_dict={self.x: dat[0],
                                                    self.y: dat[1],
                                                    self.training: False})

        print('\tVALIDATION\tLOSS: {:.10f}\tACC: {:.10f}'.format(loss, acc))
    
    def close(self):
        self.sess.close()
        

In [16]:
m.close()

TypeError: close() takes 0 positional arguments but 1 was given

In [9]:
m = Model(config, config_tf, 1e-3)

(?, 126, 126, 32)
(?, 124, 124, 32)
(?, 61, 61, 32)
(?, 29, 29, 64)
(?, 13, 13, 64)
(?, 5, 5, 64)
(?, 1600)
(?, 128)


In [10]:
dat = read_train_test(DATA)

In [12]:
m.train(dat['train'], dat['test'], 100)

EP:   0	LOSS: 1.4244050980	ACC: 0.8503254056
EP:   1	LOSS: 1.4169538021	ACC: 0.8633405566
EP:   2	LOSS: 1.4444649220	ACC: 0.8373101950
EP:   3	LOSS: 1.4429585934	ACC: 0.8481561542
EP:   4	LOSS: 1.4298957586	ACC: 0.8481561542
EP:   5	LOSS: 1.4400215149	ACC: 0.8264642358
EP:   6	LOSS: 1.4013578892	ACC: 0.8763557673
EP:   7	LOSS: 1.4075146914	ACC: 0.8633405566
EP:   8	LOSS: 1.4176148176	ACC: 0.8546637893
EP:   9	LOSS: 1.4195505381	ACC: 0.8546637893
EP:  10	LOSS: 1.4132559299	ACC: 0.8611713648
	VALIDATION	LOSS: 1.4631863832	ACC: 0.8166666627
EP:  11	LOSS: 1.4003722668	ACC: 0.8741865754
EP:  12	LOSS: 1.4082163572	ACC: 0.8698481321
EP:  13	LOSS: 1.4030748606	ACC: 0.8850325346
EP:  14	LOSS: 1.4009544849	ACC: 0.8806941509
EP:  15	LOSS: 1.4130227566	ACC: 0.8763557673
EP:  16	LOSS: 1.3840453625	ACC: 0.8937093019
EP:  17	LOSS: 1.3946684599	ACC: 0.8828633428
EP:  18	LOSS: 1.4043464661	ACC: 0.8763557673
EP:  19	LOSS: 1.4015620947	ACC: 0.8785249591
EP:  20	LOSS: 1.3929735422	ACC: 0.8872017264
	VALID